In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from datetime import datetime
import torch.nn.functional as F

In [ ]:
transform_train = torchvision.transforms.Compose([
  transforms.RandomCrop(32,padding = 4),
  torchvision.transforms.RandomHorizontalFlip(p = 0.5),
  torchvision.transforms.RandomAffine(0,translate=(0.1,0.1)),
  transforms.ToTensor(),
                                                  
]
    
)

In [ ]:
 train_dataset= torchvision.datasets.CIFAR10(
    root = '.',
    train = True,
    transform = transform_train,
    download = True
)

Files already downloaded and verified


In [ ]:

train_dataset.data.shape

(50000, 32, 32, 3)

In [ ]:
test_dataset= torchvision.datasets.CIFAR10(
    root = '.',
    train = False,
    transform = transforms.ToTensor(),
    download = True
)

Files already downloaded and verified


In [ ]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(dataset= train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset= test_dataset,batch_size=batch_size,shuffle=False)
k = 10

In [ ]:
class CNN(nn.Module):
  def __init__(self,k):
    super(CNN, self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(3 , 32, kernel_size=3,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.Conv2d(in_channels= 32 , out_channels=32, kernel_size=3,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(2)
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(in_channels= 32 , out_channels=64, kernel_size=3,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.Conv2d(in_channels= 64 , out_channels=64, kernel_size=3,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(2)

    )
    self.conv3 = nn.Sequential(
        nn.Conv2d(in_channels= 64 , out_channels=128, kernel_size=3,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(128),
        nn.Conv2d(in_channels= 128 , out_channels=128, kernel_size=3,padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(128),
        nn.MaxPool2d(2)

    )  
    self.fc1 = nn.Linear(128*4*4,1024)
    self.fc2 = nn.Linear(1024,k)
  def forward(self,X):
    x = self.conv1(X)
    x = self.conv2(X)
    x = self.conv3(X)
    print(x.size(0))
    x = x.view(x.size(0),-1)
    x = F.dropout(x, p =0.5)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, p =0.2)
    x = self.fc2(x)
    return out


In [ ]:
model = CNN(10)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu' )
print(device)
model.to(device)

cuda:0


CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel

In [ ]:
def Batch_Gd(model,criterion, optimizer, train_loader, test_loader, epoch):
  train_losses = []
  test_losses = []
  for it in range(epoch):
    t0 = datetime.now()
    train_loss = []
    for inputs, targets in train_loader:
      #mode data to gpu
      inputs, targets = inputs.to(device), targets.to(device)
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs,targets)
      loss.backward()
      optimizer.step()
      train_loss.append(loss.item())
    train_loss = np.mean(train_loss)
    test_loss = [ ]
    for inputs, targets in test_loader:
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = model(inputs)
      loss = criterion(outputs,targets)
      test_loss.append(loss.item())
    test_loss = np.mean(test_loss)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    dt = datetime.now() - t0
    print(f'Epoch {it+1}/{epoch},train loss :{train_losses[it]},test loss:{test_losses[it]}, duration = {dt}')  
  return train_losses, test_losses

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
train_losses, test_losses = Batch_Gd(model,criterion,optimizer,train_loader,test_loader,5)

RuntimeError: ignored